In [ ]:
!pip install pandas influxdb-client

In [ ]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd


# InfluxDB Connection Details
INFLUXDB_URL = "https://influxdb-battery-demo.apps.sno.pemlab.rdu2.redhat.com/"
INFLUXDB_TOKEN = "admin_token"
INFLUXDB_ORG = "redhat"
INFLUXDB_BUCKET = "bms"

In [ ]:
# Initialize Client
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)


def retrieve_battery_data():
    query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -1h)
      |> filter(fn: (r) => r["_measurement"] == "battery_data")
    '''
    query_api = client.query_api()
    tables = query_api.query(query, org=INFLUXDB_ORG)

    # Process Results
    data = []
    for table in tables:
        for record in table.records:
            data.append(record.values)

    df = pd.DataFrame(data)
    print(df)  # Print the retrieved data
    return df

In [ ]:
df = retrieve_battery_data()

# Drop non-numeric columns
df = df.drop(columns=["result", "table", "_measurement"])

# Save for ML
df.to_csv("./data/battery_raw.csv", index=False)
print("Saved raw battery data from InfluxDB.")